# Controls

> Width slider, scale slider, and card count selector components.

In [ ]:
#| default_exp components.controls

In [ ]:
#| export
from typing import Any

from fasthtml.common import Div, Span, Input, Select, Option

# DaisyUI components
from cjm_fasthtml_daisyui.components.data_input.range_slider import range_dui, range_sizes
from cjm_fasthtml_daisyui.components.data_input.select import select, select_sizes
from cjm_fasthtml_daisyui.utilities.semantic_colors import text_dui

# Tailwind utilities
from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import (
    flex_display, items, gap, grow
)
from cjm_fasthtml_tailwind.utilities.sizing import w
from cjm_fasthtml_tailwind.utilities.typography import font_size
from cjm_fasthtml_tailwind.core.base import combine_classes

# Local imports
from cjm_fasthtml_card_stack.core.config import CardStackConfig
from cjm_fasthtml_card_stack.core.html_ids import CardStackHtmlIds

## render_width_slider

Range slider for adjusting the card stack viewport width. The `oninput` handler
references the namespaced JS function `window.cardStacks[prefix].updateWidth()`.

In [ ]:
#| export
def render_width_slider(
    config: CardStackConfig,  # Card stack configuration
    ids: CardStackHtmlIds,  # HTML IDs for this instance
    card_width: int = 80,  # Current card width in rem
) -> Any:  # Width slider component
    """Render the card stack width slider control."""
    js_fn = f"window.cardStacks['{config.prefix}'].updateWidth(this.value)"

    return Div(
        Span(
            "Narrow",
            cls=combine_classes(font_size.xs, text_dui.base_content.opacity(50))
        ),
        Input(
            type="range",
            id=ids.width_slider,
            min=str(config.card_width_min),
            max=str(config.card_width_max),
            step=str(config.card_width_step),
            value=str(card_width),
            cls=combine_classes(range_dui, range_sizes.xs, grow()),
            oninput=js_fn
        ),
        Span(
            "Wide",
            cls=combine_classes(font_size.xs, text_dui.base_content.opacity(50))
        ),
        cls=combine_classes(flex_display, items.center, gap(3), w.full)
    )

In [ ]:
# Test render_width_slider
from fasthtml.common import to_xml
from cjm_fasthtml_card_stack.core.config import CardStackConfig, _reset_prefix_counter

_reset_prefix_counter()
config = CardStackConfig(prefix="test")
ids = CardStackHtmlIds(prefix="test")

slider = render_width_slider(config, ids, card_width=60)
html = to_xml(slider)
assert 'id="test-width-slider"' in html
assert 'min="30"' in html
assert 'max="120"' in html
assert 'step="5"' in html
assert 'value="60"' in html
assert "window.cardStacks['test'].updateWidth" in html
assert "Narrow" in html
assert "Wide" in html
print("render_width_slider tests passed!")

render_width_slider tests passed!


## render_scale_slider

Range slider for adjusting card content scale. Works the same way as the
width slider but controls the `card_scale` percentage.

In [ ]:
#| export
def render_scale_slider(
    config: CardStackConfig,  # Card stack configuration
    ids: CardStackHtmlIds,  # HTML IDs for this instance
    card_scale: int = 100,  # Current scale percentage
) -> Any:  # Scale slider component
    """Render the card stack scale slider control."""
    js_fn = f"window.cardStacks['{config.prefix}'].updateScale(this.value)"

    return Div(
        Span(
            "Smaller",
            cls=combine_classes(font_size.xs, text_dui.base_content.opacity(50))
        ),
        Input(
            type="range",
            id=ids.scale_slider,
            min=str(config.card_scale_min),
            max=str(config.card_scale_max),
            step=str(config.card_scale_step),
            value=str(card_scale),
            cls=combine_classes(range_dui, range_sizes.xs, grow()),
            oninput=js_fn
        ),
        Span(
            "Larger",
            cls=combine_classes(font_size.xs, text_dui.base_content.opacity(50))
        ),
        cls=combine_classes(flex_display, items.center, gap(3), w.full)
    )

In [ ]:
# Test render_scale_slider
slider = render_scale_slider(config, ids, card_scale=150)
html = to_xml(slider)
assert 'id="test-scale-slider"' in html
assert 'min="50"' in html
assert 'max="200"' in html
assert 'step="10"' in html
assert 'value="150"' in html
assert "window.cardStacks['test'].updateScale" in html
assert "Smaller" in html
assert "Larger" in html
print("render_scale_slider tests passed!")

render_scale_slider tests passed!


## render_card_count_select

Dropdown selector for changing the number of visible cards in the viewport.
Triggers a full viewport re-render via the namespaced JS function.

In [ ]:
#| export
def render_card_count_select(
    config: CardStackConfig,  # Card stack configuration
    ids: CardStackHtmlIds,  # HTML IDs for this instance
    current_count: int = 3,  # Currently selected card count
    is_auto_mode: bool = False,  # Whether auto-adjust mode is active
) -> Any:  # Card count dropdown component
    """Render the card count dropdown selector."""
    js_fn = f"window.cardStacks['{config.prefix}'].handleCountChange(this.value)"

    options = []

    # Auto option — selected if is_auto_mode is True
    if config.auto_visible_count:
        options.append(Option("Auto", value="auto", selected=is_auto_mode))

    # Numeric options — only selected if NOT in auto mode
    options.extend(
        Option(
            f"{count} card{'s' if count > 1 else ''}",
            value=str(count),
            selected=(not is_auto_mode and count == current_count)
        )
        for count in config.visible_count_options
    )

    return Select(
        *options,
        id=ids.card_count_select,
        cls=combine_classes(select, select_sizes.sm),
        onchange=js_fn
    )

In [ ]:
# Test render_card_count_select
select_el = render_card_count_select(config, ids, current_count=5)
html = to_xml(select_el)
assert 'id="test-card-count-select"' in html
assert "window.cardStacks['test'].handleCountChange" in html
assert 'value="auto"' in html
assert "Auto" in html
assert "1 card" in html
assert "3 cards" in html
assert "5 cards" in html
assert "7 cards" in html
assert "9 cards" in html
# By default, is_auto_mode=False, so numeric option should be selected
assert 'value="5" selected' in html
print("render_card_count_select tests passed!")

render_card_count_select tests passed!


In [ ]:
# Test with custom visible_count_options
custom_config = CardStackConfig(prefix="custom", visible_count_options=(3, 5, 7))
custom_ids = CardStackHtmlIds(prefix="custom")
select_el = render_card_count_select(custom_config, custom_ids, current_count=3)
html = to_xml(select_el)
assert "1 card" not in html  # Not in custom options
assert "3 cards" in html
assert "9 cards" not in html  # Not in custom options
assert "Auto" in html  # Auto still present by default

# Test auto_visible_count=False hides Auto option
no_auto_config = CardStackConfig(prefix="noauto", auto_visible_count=False)
no_auto_ids = CardStackHtmlIds(prefix="noauto")
select_el = render_card_count_select(no_auto_config, no_auto_ids, current_count=3)
html = to_xml(select_el)
assert 'value="auto"' not in html
assert "Auto" not in html
print("Custom options test passed!")

# Test is_auto_mode=True selects Auto option
select_auto = render_card_count_select(config, ids, current_count=5, is_auto_mode=True)
html_auto = to_xml(select_auto)
assert 'value="auto" selected' in html_auto  # Auto should be selected
assert 'value="5" selected' not in html_auto  # Numeric should NOT be selected
print("is_auto_mode=True test passed!")

Custom options test passed!
is_auto_mode=True test passed!


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()